# MTMount accelerometers
Querying MTMount accelerometers \
Craig Lage - 22-Mar-23

In [ ]:
import sys, time, os, asyncio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient, merge_packed_time_series

In [ ]:
import lsst_efd_client
#os.path.abspath(lsst_efd_client.__file__)

In [ ]:
# Get EFD client
client = EfdClient('usdf_efd')

In [ ]:
# Times to look at the data
start = Time("2025-04-09 14:40:00Z", scale='utc')
end = Time("2025-04-09 14:55:00Z", scale='utc')

In [ ]:
baseFields = ['accelerationX', 'accelerationY', 'accelerationZ']
sensorNames = ['SST top end ring +x -y', 'SST top end ring -x -y', 'SST spider spindle']#, 'SST M2 surrogate'] 

##  Get all of the data for the selected times

In [ ]:
packed_dataframe = await client.select_time_series("lsst.sal.ESS.accelerometer", ["*"], start, end)

## Now unpack it and plot it by sensor
### The pink warnings you'll see are due to an update that needs to be fixed in lsst_efd_client.
### Note that the one labeled "SST M2 Surrogate" does not appear to be connected, so this data is meaningless.

In [ ]:
fig, axs = plt.subplots(2,2, figsize = (8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for i, sensorName in enumerate(sensorNames):
    sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName==sensorName]
    plotx = i%2
    ploty = int(i/2)
    ax = axs[plotx][ploty]
    ax.set_title(sensorName)
    ax.set_ylabel("Accel(m/s^2)")
    for baseField in baseFields:
        df = merge_packed_time_series(sub_dataframe, baseField, stride=1,
                             ref_timestamp_col="timestamp", fmt='unix_tai',
                             scale='tai')
        df[baseField].plot(ax=ax, label=list(baseField)[-1])
        #df[baseField].plot(ax=ax, label=list(baseField)[-1])
    ax.legend()
    #ax.set_ylim(-0.05, 0.05)
plt.savefig("/home/c/cslage/u/MTMount/accel_data/Azimuth_Slews_09Apr25.png")

## With low pass filter

In [ ]:
from scipy.signal import butter,filtfilt

def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

fig, axs = plt.subplots(2,2, figsize = (8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for i, sensorName in enumerate(sensorNames[1:2]):
    sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName==sensorName]
    plotx = i%2
    ploty = int(i/2)
    ax = axs[plotx][ploty]
    ax.set_title(sensorName)
    ax.set_ylabel("Accel(m/s^2)")
    for baseField in baseFields[2:3]:
        df = merge_packed_time_series(sub_dataframe, baseField, stride=1,
                             ref_timestamp_col="timestamp", fmt='unix_tai',
                             scale='tai')
        plotDF = df[baseField] - df[baseField].mean()
        plotDF = plotDF.rolling(200).sum() / 200.0
        values = plotDF.values[200:-1]
        times = df['times'].values[200:-1]
        filtered_values = butter_lowpass_filter(values, 1.0, 200, 3)
        plt.plot(times, values) 
        #df[baseField].plot(ax=ax, label=list(baseField)[-1])
    ax.legend()
    #ax.set_ylim(-0.05, 0.05)
plt.savefig("/home/c/cslage/u/MTMount/accel_data/Azimuth_Slews_Filtered_10Apr25.png")